<a href="https://colab.research.google.com/github/aksharakandimalla/Convolutional-Neural-Networks/blob/master/cats_vs_dogs(transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
                                        input_shape = (150,150,3),
                                        include_top = False,
                                        weights = None

)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

#pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('the shape of the last layer is', last_layer.output_shape)
last_output = last_layer.output

--2020-08-11 12:25:38--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  85.1MB/s    in 1.0s    

2020-08-11 12:25:39 (85.1 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

the shape of the last layer is (None, 7, 7, 768)


In [15]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1,activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)
model.compile(optimizer = RMSprop(lr = 0.0001), loss = 'binary_crossentropy', metrics = 'accuracy')


In [16]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

--2020-08-11 12:57:43--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.28.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.28.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  92.6MB/s    in 0.7s    

2020-08-11 12:57:44 (92.6 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [22]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

In [27]:
train_datagen = ImageDataGenerator(
                                      rescale = 1./255,
                                      rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      zoom_range = 0.2,
                                      shear_range = 0.2,
                                      horizontal_flip = True
)

train_generator = train_datagen.flow_from_directory(
                                                      train_dir,
                                                      batch_size = 20,
                                                      class_mode = 'binary',
                                                      target_size = (150,150))

validation_datagen = ImageDataGenerator(
                                      rescale = 1./255
)
validation_generator = validation_datagen.flow_from_directory(
                                                      validation_dir,
                                                      batch_size = 20,
                                                      class_mode = 'binary',
                                                      target_size = (150,150)
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [33]:
history = model.fit_generator(
                                    train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 20, validation_steps = 50, verbose = 2
)

Epoch 1/20
100/100 - 152s - loss: 0.2368 - accuracy: 0.9105 - val_loss: 0.0958 - val_accuracy: 0.9590
Epoch 2/20
100/100 - 154s - loss: 0.1807 - accuracy: 0.9295 - val_loss: 0.0982 - val_accuracy: 0.9640
Epoch 3/20
100/100 - 151s - loss: 0.1883 - accuracy: 0.9295 - val_loss: 0.1175 - val_accuracy: 0.9650
Epoch 4/20
100/100 - 151s - loss: 0.1816 - accuracy: 0.9425 - val_loss: 0.1766 - val_accuracy: 0.9580
Epoch 5/20
100/100 - 150s - loss: 0.1999 - accuracy: 0.9305 - val_loss: 0.1052 - val_accuracy: 0.9620
Epoch 6/20
100/100 - 154s - loss: 0.1849 - accuracy: 0.9320 - val_loss: 0.1187 - val_accuracy: 0.9700
Epoch 7/20
100/100 - 151s - loss: 0.1728 - accuracy: 0.9350 - val_loss: 0.1730 - val_accuracy: 0.9520
Epoch 8/20
100/100 - 151s - loss: 0.1473 - accuracy: 0.9545 - val_loss: 0.1368 - val_accuracy: 0.9590
Epoch 9/20
100/100 - 151s - loss: 0.1767 - accuracy: 0.9440 - val_loss: 0.1375 - val_accuracy: 0.9590
Epoch 10/20
100/100 - 154s - loss: 0.1463 - accuracy: 0.9520 - val_loss: 0.1591 - 

In [34]:
import matplotlib.pyplot as plt

test_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
test_loss = history.history['loss']
validation_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, test_accuracy, 'r', label = 'test-accuracy')
plt.plot(epochs, validation_accuracy, 'b', label = 'val-accuracy')
plt.title('accuracy comparisson')
plt.legend(loc = 0)
plt.figure()
plt.show()

NameError: ignored